<a href="https://colab.research.google.com/github/aysenurcftc/Fine-tuning_LLMs/blob/main/LLM_feature_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LLM as Feature Extractor

https://github.com/rasbt/LLM-finetuning-scripts/blob/main/conventional/distilbert-movie-review/1_feature-extractor.ipynb

https://magazine.sebastianraschka.com/p/finetuning-large-language-models

In [112]:
#!pip install transformers datasets

In [123]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [124]:
from datasets import load_dataset

ds = load_dataset("legacy107/spamming-email-classification")
ds = ds.rename_columns({"Text": "text", "Spam": "label"})

"""

ds_small = DatasetDict({
    "train": ds["train"].shuffle(seed=42).select(range(1000)),
    "validation": ds["validation"].shuffle(seed=42).select(range(200)),
    "test": ds["test"].shuffle(seed=42).select(range(200)),
})

"""

'\n\nds_small = DatasetDict({\n    "train": ds["train"].shuffle(seed=42).select(range(1000)),\n    "validation": ds["validation"].shuffle(seed=42).select(range(200)),\n    "test": ds["test"].shuffle(seed=42).select(range(200)),\n})\n\n'

In [125]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4556
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 569
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 570
    })
})

In [126]:
ds['train'][0]

{'text': 'Subject: aiesec polska - eurolds 2000  jarek ,  czy enron moze pomoc w organizacji tej imprezy ?  bylaby to dobra okazja nawiazania wielu pozytecznych kontaktow .  wicek  - - - - - - - - - - - - - - - - - - - - - - forwarded by vince j kaminski / hou / ect on 02 / 17 / 2000  08 : 51 am - - - - - - - - - - - - - - - - - - - - - - - - - - -  " andrzej wodnicki " on 02 / 16 / 2000 02 : 50 : 05 am  to : vince j kaminski / hou / ect @ ect  cc :  subject : aiesec polska - eurolds 2000  szanowny panie kaminski !  nazywam sie andrzej wodnicki i jestem czlonkiem stowarzysznia studentow  aiesec przy szkole glownej handlowej ( dawnej sgpis ) .  prosze o poswiecenie paru chwil na przeczytanie tego maila .  ( kontakt do pana otrzymalem od kolegi , ktory organizowal prezentacje firmy  enron na sgh , a posrednio od pana jarka astramowicza , przedstawiciela enron  na polske . )  w imieniu aiesec polska chcialbym zwrocic sie do pana z wielka prosba pomocy  przy wydarzeniu , ktore w tym roku o

In [127]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print("Tokenizer input max length:", tokenizer.model_max_length)
print("Tokenizer vocabulary size:", tokenizer.vocab_size)

Tokenizer input max length: 512
Tokenizer vocabulary size: 30522


In [128]:
def tokenize_text(batch):
    return tokenizer(batch["text"], truncation=True, padding=True)

In [129]:
email_tokenized = ds.map(tokenize_text, batched=True, batch_size=None)

In [130]:
email_tokenized

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4556
    })
    val: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 569
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 570
    })
})

In [131]:
from transformers import AutoModel

model = AutoModel.from_pretrained("distilbert-base-uncased")
model.to(device)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [132]:
email_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [133]:
test_batch = {"attention_mask": email_tokenized["train"][:3]["attention_mask"].to(device),
              "input_ids": email_tokenized["train"][:3]["input_ids"].to(device)}

with torch.inference_mode():
    test_output = model(**test_batch)

test_output.last_hidden_state.shape

torch.Size([3, 512, 768])

In [134]:
cls_token_output = test_output.last_hidden_state[:, 0]
cls_token_output.shape

torch.Size([3, 768])

In [135]:
@torch.inference_mode()
def get_output_embeddings(batch):
    output = model(
        batch["input_ids"].to(device),
        attention_mask=batch["attention_mask"].to(device)).last_hidden_state[:, 0]
    return {"features": output.cpu().numpy()}

In [136]:
import time
start = time.time()

email_features = email_tokenized.map(get_output_embeddings, batched=True, batch_size=10)

Map:   0%|          | 0/570 [00:00<?, ? examples/s]

In [137]:
import numpy as np

X_train = np.array(email_features["train"]["features"])
y_train = np.array(emotion_features["train"]["label"])

X_val = np.array(email_features["val"]["features"])
y_val = np.array(email_features["val"]["label"])

X_test = np.array(email_features["test"]["features"])
y_test = np.array(email_features["test"]["label"])

### Train Model on Embeddings (Extracted Features)

In [138]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

print("Training accuracy", clf.score(X_train, y_train))
print("Validation accuracy", clf.score(X_val, y_val))
print("test accuracy", clf.score(X_test, y_test))

end = time.time()
elapsed = end - start
print(f"Time elapsed {elapsed/60:.2f} min")

Training accuracy 0.9949517120280948
Validation accuracy 0.9912126537785588
test accuracy 0.9912280701754386
Time elapsed 0.44 min


In [139]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

print("Training accuracy", clf.score(X_train, y_train))
print("Validation accuracy", clf.score(X_val, y_val))
print("test accuracy", clf.score(X_test, y_test))

Training accuracy 1.0
Validation accuracy 0.9578207381370826
test accuracy 0.9736842105263158
